In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.31 already installed.


## Download dataset

https://open-bitbucket.nrao.edu/projects/CASA/repos/casatestdata/browse/unittest/tclean/venus_ephem_test.ms

In [2]:
import graphviper

graphviper.utils.data.download(file="sdimaging.ms")

[2024-07-18 14:50:19,134]  WARNING  graphviper:  File exists: /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox 
[2024-07-18 14:50:19,134]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List  
 ─────────────── 
  sdimaging.ms

[2024-07-18 14:50:19,850]     INFO  graphviper:  File exists: sdimaging.ms 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "sdimaging.ms"

main_chunksize = {"time": 63}  # baseline, polarization
outfile = "sdimaging.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-07-18 14:50:20,057]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'FIELD_ID'] 
[2024-07-18 14:50:20,060]     INFO  graphviper:  Number of partitions: 1 
[2024-07-18 14:50:20,060]     INFO  graphviper:  DDI [0], STATE [0], FIELD [0], SCAN [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61] 


## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

outfile = "sdimaging.vis.zarr"
ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,spw_id,field_id,field_name,source_id,source_name,field_coords,start_frequency,end_frequency
0,sdimaging_0,"OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG","(3843, 1, 1024, 2)","[XX, YY]",0,[0],[FLS3a],[0],[FLS3a],"[fk5, 0h00m00.00s, 0d00m00.00s]",1.419395e+09,1.421893e+09


In [5]:
ps.keys()

dict_keys(['sdimaging_0'])

In [6]:
ps["sdimaging_0"]

<xarray.Dataset> Size: 71MB
Dimensions:                     (time: 3843, baseline_id: 1, frequency: 1024,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 4B dask.array<chunksize=(1,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 8B 0
  * frequency                   (frequency) float64 8kB 1.419e+09 ... 1.422e+09
  * polarization                (polarization) <U2 16B 'XX' 'YY'
  * time                        (time) float64 31kB 1.069e+09 ... 1.069e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 31kB dask.array<chunksize=(63, 1), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 8MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    SPECTRUM                    (time, baseline_id, frequency, polarization) float32 31MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 31kB dask.array<chunksize=(63, 1), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 92kB dask.array<chunksize=(63, 1, 3), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 31MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'spectrum': 'SPECTRUM', 'uvw':...
    partition_info:  {'field_id': [0], 'field_name': ['FLS3a'], 'obs_mode': '...
    pointing_xds:    <xarray.Dataset> Size: 154kB\nDimensions:        (time: ...
    weather_xds:     <xarray.Dataset> Size: 56B\nDimensions:         (station...
    antenna_xds:     <xarray.Dataset> Size: 152B\nDimensions:        (antenna...

In [7]:
ps["sdimaging_0"].attrs["antenna_xds"]

<xarray.Dataset> Size: 152B
Dimensions:        (antenna_id: 1, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 8B 0
    mount          (antenna_id) <U6 24B dask.array<chunksize=(1,), meta=np.ndarray>
    name           (antenna_id) <U3 12B dask.array<chunksize=(1,), meta=np.ndarray>
    station        (antenna_id) <U9 36B dask.array<chunksize=(1,), meta=np.ndarray>
    type           (antenna_id) <U1 4B dask.array<chunksize=(1,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 24B dask.array<chunksize=(1, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 24B dask.array<chunksize=(1, 3), meta=np.ndarray>